In [3]:
# %%
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset

import random
import os
import numpy as np

from torch.autograd import Variable
from utils import *

from transformers import AutoModelForSequenceClassification
from transformers import AutoTokenizer
import logging    # first of all import the module
from datetime import datetime
from transformers import RobertaConfig, RobertaForSequenceClassification

model =AutoModelForSequenceClassification.from_pretrained('roberta-base',num_labels= 3) 


Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.dense.bias', 'lm_head.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'roberta.pooler.dense.bias', 'lm_head.layer_norm.bias', 'roberta.pooler.dense.weight', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.out_proj.weight', 'classi

In [9]:
import torch
x = torch.tensor([1, 2, 3])
print(x.shape)
x.repeat(4,2,2).shape


torch.Size([3])


torch.Size([4, 2, 6])